In [111]:
import pandas as pd
import numpy as np
  
from string import punctuation
full_punctuation = punctuation + "–" + "," + "»" + "«" + "…" +'’'

In [86]:
from gensim.models.wrappers import FastText
model_fr = FastText.load_fasttext_format('/Users/nigula/input/fr/fr.bin')

In [47]:
from gensim.models.keyedvectors import FastTextKeyedVectors


In [76]:
from  sklearn.metrics.pairwise import cosine_similarity


In [48]:
model_rus = FastTextKeyedVectors.load('/Users/nigula/input/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model')


# one_element evaluation

In [10]:
reference_dict = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/reference_dict.csv")
reference_dict.head()

,English,Russian,French
0,letter,письмо,lettre
1,letter,буква,lettre
2,letter,буква,caractère
3,plate,тарелка,assiette
4,plate,тарелка,plat


In [52]:
def update_dict (ref_dict_eng_word, l1_w, l2_w):
    if l1_w in ref_dict_eng_word:
        ref_dict_eng_word[l1_w].append((l1_w, l2_w))
    else:
        ref_dict_eng_word[l1_w] = []
        ref_dict_eng_word[l1_w].append((l1_w, l2_w))

In [53]:
def reference_to_dict (df):
    ref_dict = {}
    for df_ind, val in df.iterrows():
        #print(list(val))
        
        val_list = list(val)
        eng_word = val_list[0]
        l1_word = val_list[1] 
        l2_word = val_list[2]
        if eng_word in ref_dict:
            update_dict(ref_dict[eng_word],l1_word, l2_word)
            #ref_dict[eng_word].append((l1_word, l2_word))
        else:
            ref_dict[eng_word] = {}
            update_dict(ref_dict[eng_word],l1_word, l2_word)
            #ref_dict[eng_word].append((l1_word, l2_word))
        #break
    return ref_dict
    
ref_dct = reference_to_dict(reference_dict)

In [158]:
ref_dct['necessary']

{'необходимый': [('необходимый', 'nécessaire')]}

In [35]:
merge_cont = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/systran_merge_mean_04_similarity_vs_pos.csv")

In [36]:
merge_cont.head()

,Unnamed: 0,eng_word,l1,l1_context,l2,l2_context
0,0,treat,gâterie,wholesome treat|sweet treats|,угощение,morning treat|feast |
1,1,treat,gâterie,wholesome treat|sweet treats|,удовольствие,pleasure | fun | satisfaction | enjoyment | am...
2,2,treat,soigner,to treat people|to treat angina|cure | heal | ...,лечить,to treat people|heal | cure | heal | dose |
3,3,treat,soigner,to treat people|to treat angina|cure | heal | ...,вылечить,cure | heal |
4,4,treat,considérer,treated on a case-by-case basis|treated confid...,рассматривать,consider | see | view | regard | examine | rev...


In [38]:
merge_cont_short = pd.DataFrame()
merge_cont_short['en'] = merge_cont['eng_word']
merge_cont_short['rus'] = merge_cont['l2']
merge_cont_short['fr'] = merge_cont['l1']
merge_cont_short.head()


,en,rus,fr
0,treat,угощение,gâterie
1,treat,удовольствие,gâterie
2,treat,лечить,soigner
3,treat,вылечить,soigner
4,treat,рассматривать,considérer


In [55]:
systran_merge_mean_04_similarity_vs_pos_dict = reference_to_dict(merge_cont_short)

In [128]:
systran_merge_mean_04_similarity_vs_pos_dict['star']

{'звезда': [('звезда', 'étoile'),
  ('звезда', 'star'),
  ('звезда', 'vedette'),
  ('звезда', 'étoile'),
  ('звезда', 'star'),
  ('звезда', 'vedette')],
 'стар': [('стар', 'étoile'), ('стар', 'étoile')]}

In [129]:
ref_dct['star']

{'звезда': [('звезда', 'étoile'), ('звезда', 'vedette'), ('звезда', 'star ')],
 'звезда (первый ученик)': [('звезда (первый ученик)', 'meilleur')],
 'играть главную роль': [('играть главную роль', 'être la vedette'),
  ('играть главную роль', 'tenir le rôle principal')]}

In [73]:
a = ('пауза', 'no_equality')
'no_equality' in a

True

In [116]:
#l1 = "maman"
#l2 = 'mère de famille'
l1 = 'поломка, скол'
l2 = 'сломать'
 

def get_vectors_sum(lemm_line, model):
    orig_vector = np.zeros(shape = 300)
    for lemma in lemm_line.split():
        #print(lemma)
        try:
            orig_vector += model[lemma]
        except:
            pass
    return orig_vector

def find_lines_similarity(line_1, line_2, model):
    line_1_clear = ''
    for line_1_char in line_1:
        if line_1_char not in full_punctuation:
            line_1_clear += line_1_char
    line_2_clear = ''
    for line_2_char in line_2:
        if line_2_char not in full_punctuation:
            line_2_clear += line_2_char
    l1_vect = get_vectors_sum(line_1_clear, model)
    l2_vect = get_vectors_sum(line_2_clear, model)
    cos_sim = cosine_similarity(l1_vect.reshape(1, -1), l2_vect.reshape(1, -1))[0][0]
    #print(cos_sim)
    return cos_sim

find_lines_similarity(l1, l2, model_rus)    
    

0.3162484806225587

In [135]:
def eval_pair_similarity(pair_1, pair_2):
    if 'no_equality' in pair_1 or 'no_equality' in pair_2:
        return False
    else:
        l1_word_p1 = pair_1[0]
        l1_word_p2 = pair_2[0]
        #print(l1_word_p1, l1_word_p2)
        
        l2_word_p1 =  pair_1[1]
        l2_word_p2 = pair_2[1]
        #print(l2_word_p1, l2_word_p2)
        l1_words_sim = find_lines_similarity(l1_word_p1, l1_word_p2, model_rus)
        if l1_words_sim > 0.6:
            l2_words_sim = find_lines_similarity(l2_word_p1, l2_word_p2, model_fr)
            if l2_words_sim > 0.8:
                return True
    return "pisolse"
            
#eval_pair_similarity(('пауза', 'pause '),('пауза', 'no_equality') )
eval_pair_similarity(('перерыв', 'vacances'), ('перерыв', 'pause'))

'pisolse'

In [171]:
def evaluate_eng_word_quality(ref_dict_eng_word, ev_dict_eng_word, debug = False):
    all_var_reference_pairs = []
    for l1_word in ref_dict_eng_word.keys():
        for ref_pair in ref_dict_eng_word[l1_word]:
            ref_pair_line = ref_pair[0] + '_' + ref_pair[1]
            all_var_reference_pairs.append(ref_pair_line)
    all_var_reference_pairs_set = set(all_var_reference_pairs)
    all_var_ev_dict_pairs = []
    for l1_word_ev in ev_dict_eng_word.keys():
        for ev_pair in ev_dict_eng_word[l1_word_ev]:
            eval_pair_line = ev_pair[0] + "_" + ev_pair[1]
            all_var_ev_dict_pairs.append(eval_pair_line)
    all_var_ev_dict_pairs_set = set(all_var_ev_dict_pairs)
    
    handled_reference_pairs = []
    handled_ev_dict_pairs = []
    #get direct l1 word intersections
    for l1_word in ref_dict_eng_word.keys():
        if l1_word in ev_dict_eng_word:
            print("***********",l1_word,"***********")
            if debug == True: print(ref_dict_eng_word[l1_word], ev_dict_eng_word[l1_word])
            for ref_pair in ref_dict_eng_word[l1_word]:
                if debug == True: print(ref_pair)
                for eval_pair in ev_dict_eng_word[l1_word]:
                    if debug == True:print(ref_pair, eval_pair)
                    pair_sim_res = eval_pair_similarity(ref_pair, eval_pair)
                    if pair_sim_res == True:
                        if debug == True:print("positive_abs_eq")
                        ref_pair_line = ref_pair[0] + '_' + ref_pair[1]
                        handled_reference_pairs.append(ref_pair_line)
                        eval_pair_line = eval_pair[0] + "_" + eval_pair[1]
                        handled_ev_dict_pairs.append(eval_pair_line)
                    else:
                        if debug == True: print("NO_ABS_EQ")
                        pass
        else:
            print(l1_word, "not in eval dict")
            
    print("=======DIRECT SIM FINISHED========") 
    print(handled_reference_pairs, handled_ev_dict_pairs)
    #get w2v similar l1 words_intersect
    eval_l1_words_list = list(ev_dict_eng_word.keys())
    for l1_word_ref in ref_dict_eng_word.keys():
        if l1_word_ref not in ev_dict_eng_word:
            for l1_word_eval in eval_l1_words_list:
                
                l1_word_sim = find_lines_similarity(l1_word_ref,l1_word_eval, model_rus)
                print(l1_word_ref,l1_word_eval,l1_word_sim)
                if l1_word_sim > 0.5:
                    print("l1_word_sim worked")
                    for ref_pair in ref_dict_eng_word[l1_word_ref]:
                        for eval_pair in ev_dict_eng_word[l1_word_eval]:
                            print(ref_pair, eval_pair)
                            pair_sim_res = eval_pair_similarity(ref_pair, eval_pair)
                            if pair_sim_res == True:
                                print("match_found")
                                ref_pair_line = ref_pair[0] + '_' + ref_pair[1]
                                handled_reference_pairs.append(ref_pair_line)
                                eval_pair_line = eval_pair[0] + "_" + eval_pair[1]
                                handled_ev_dict_pairs.append(eval_pair_line)
                            else:
                                print("no_match")
    print(handled_reference_pairs, handled_ev_dict_pairs)
    handled_reference_pairs_set = set(handled_reference_pairs)
    handled_ev_dict_pairs_set = set(handled_ev_dict_pairs)
    
    reference_intersect = handled_reference_pairs_set.intersection(all_var_reference_pairs_set)
    print(reference_intersect, len(reference_intersect)/len(all_var_reference_pairs_set))
    reference_dict_coverage = len(reference_intersect)/len(all_var_reference_pairs_set)
    
    eval_intersect = handled_ev_dict_pairs_set.intersection(all_var_ev_dict_pairs_set)
    print(eval_intersect, "eval_intersect",len(eval_intersect) , "all_var_ev_dict_pairs_set", 
          len(all_var_ev_dict_pairs_set),
          len(eval_intersect)/len(all_var_ev_dict_pairs_set), )
    print("****************\n")
    paired_eval_pairs = len(eval_intersect)/len(all_var_ev_dict_pairs_set)
    
    return reference_dict_coverage, paired_eval_pairs
    
evaluate_eng_word_quality(ref_dct['star'], systran_merge_mean_04_similarity_vs_pos_dict['star'])       

*********** звезда ***********
=======DIRECT SIM FINISHED========
['звезда_étoile', 'звезда_étoile', 'звезда_vedette', 'звезда_vedette', 'звезда_star ', 'звезда_star '] ['звезда_étoile', 'звезда_étoile', 'звезда_vedette', 'звезда_vedette', 'звезда_star', 'звезда_star']
звезда (первый ученик) звезда 0.6145184998958503
l1_word_sim worked
('звезда (первый ученик)', 'meilleur') ('звезда', 'étoile')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'star')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'vedette')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'étoile')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'star')
no_match
('звезда (первый ученик)', 'meilleur') ('звезда', 'vedette')
no_match
звезда (первый ученик) стар 0.2641785284133709
играть главную роль звезда 0.060604119962037885
играть главную роль стар -0.03183579444872556
['звезда_étoile', 'звезда_étoile', 'звезда_vedette', 'звезда_vedette', 'звезда_star ', 'звезда_star '] ['звез

(0.5, 0.75)

In [145]:
systran_merge_mean_04_similarity_vs_pos_dict['star']

{'звезда': [('звезда', 'étoile'),
  ('звезда', 'star'),
  ('звезда', 'vedette'),
  ('звезда', 'étoile'),
  ('звезда', 'star'),
  ('звезда', 'vedette')],
 'стар': [('стар', 'étoile'), ('стар', 'étoile')]}

In [172]:
def eval_dicts(ref_dict, eval_dict):
    eng_word_list = []
    ref_dict_coverage = []
    exceed_pairs_generation = []
    for ref_eng_word in ref_dict.keys():
        print(ref_eng_word)
        if ref_eng_word in eval_dict:
            cov, exc = evaluate_eng_word_quality(ref_dct[ref_eng_word], eval_dict[ref_eng_word])       
            eng_word_list.append(ref_eng_word)
            ref_dict_coverage.append(cov)
            exceed_pairs_generation.append(exc)  
    data = pd.DataFrame(list(zip(eng_word_list, ref_dict_coverage,exceed_pairs_generation)),
                        columns =['eng_word', 'reference_coverage','paired_eval_pairs'])
    return data
df = eval_dicts(ref_dct,systran_merge_mean_04_similarity_vs_pos_dict)
df.head()           

letter
plate
dinner
*********** обед ***********
*********** ужин ***********
=======DIRECT SIM FINISHED========
['ужин_dîner', 'ужин_dîner'] ['ужин_dîner', 'ужин_dîner']
['ужин_dîner', 'ужин_dîner'] ['ужин_dîner', 'ужин_dîner']
{'ужин_dîner'} 0.5
{'ужин_dîner'} eval_intersect 1 all_var_ev_dict_pairs_set 5 0.2
****************

necessary
=======DIRECT SIM FINISHED========
[] []
необходимый нужный 0.715085602657199
l1_word_sim worked
('необходимый', 'nécessaire') ('нужный', 'nécessaire')
match_found
('необходимый', 'nécessaire') ('нужный', 'indispensable')
no_match
('необходимый', 'nécessaire') ('нужный', 'requis')
no_match
('необходимый', 'nécessaire') ('нужный', 'nécessaire')
match_found
('необходимый', 'nécessaire') ('нужный', 'indispensable')
no_match
('необходимый', 'nécessaire') ('нужный', 'requis')
no_match
необходимый должный 0.5440062387986638
l1_word_sim worked
('необходимый', 'nécessaire') ('должный', 'nécessaire')
no_match
('необходимый', 'nécessaire') ('должный', 'nécessair

очко, гол no_equality -0.030650050049549828
очко, гол очки 0.7118143877789564
l1_word_sim worked
('очко, гол', 'point') ('очки', 'no_equality')
no_match
очко, гол ссылаться -0.2592275284960495
кончик, острие точка 0.27328860693272133
кончик, острие пункт -0.07186902992952095
кончик, острие момент 0.029647518328090493
кончик, острие балл -0.025057733601653507
кончик, острие дело -0.03245780458801574
кончик, острие очко 0.13683074167768713
кончик, острие смысл 0.07927725554530958
кончик, острие указывать -0.1295436039816327
кончик, острие указать 0.031932599610218604
кончик, острие направить 0.040287149040153715
кончик, острие отметить -0.023684888467373443
кончик, острие показывать -0.10625456132706437
кончик, острие обращать -0.13549093110993718
кончик, острие отмечать -0.13586246150885017
кончик, острие no_equality -0.03755355811046316
кончик, острие очки 0.126557165059394
кончик, острие ссылаться -0.23860655147946797
запятая (в числах) точка 0.16780496359844388
запятая (в числах) пун

,eng_word,reference_coverage,paired_eval_pairs
0,dinner,0.5,0.200000
1,necessary,1.0,0.166667
2,table,0.5,0.222222
3,warm,1.0,0.333333
4,entire,0.0,0.000000


In [ ]:
df['average_mark'] = 0.6 * df['reference_coverage'] + 0.4 * df['']

In [184]:
def get_overall_evaluation(eval_df):
    overall_marks = 0.6 * eval_df['reference_coverage'] + 0.4 * eval_df['paired_eval_pairs']
    print(eval_df.info())
    return np.median(overall_marks)
get_overall_evaluation(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
eng_word              16 non-null object
reference_coverage    16 non-null float64
paired_eval_pairs     16 non-null float64
dtypes: float64(2), object(1)
memory usage: 464.0+ bytes
None


0.3844444444444445

In [166]:
df.head(20)           

,eng_word,reference_coverage,exceeded_eval_pairs
0,dinner,0.500000,0.800000
1,necessary,1.000000,0.833333
2,table,0.500000,0.777778
3,warm,1.000000,0.666667
4,entire,0.000000,1.000000
5,background,0.250000,0.909091
6,child,1.000000,0.666667
7,childhood,1.000000,0.666667
8,grandchild,0.500000,0.500000
9,point,0.153846,0.923077


# AZURE CONTEX MERGE EVAL

In [186]:
AZURE_MERGE_df = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/azure_context_merge_approach.csv")


In [187]:
AZURE_MERGE_df.head()

,Unnamed: 0,eng_word,l1,l1_context,l2,l2_context
0,0,necessary,mal nécessaire,necessary |,необходимые,necessary | required | needed | essential | ap...
1,1,necessary,mal nécessaire,necessary |,нужно,need | should | want | needs | must | needed |...
2,2,necessary,mal nécessaire,necessary |,требуется,required | needed | need | takes | necessary |...
3,3,necessary,mal nécessaire,necessary |,надо,should | necessary | Nothing is necessary to m...
4,4,necessary,si nécessaire,"if necessary | necessary | If necessary, we’ll...",обязательно,necessarily | will definitely | necessary | re...


In [195]:
az_df = AZURE_MERGE_df[['eng_word', 'l2','l1']]
az_df.head()

,eng_word,l2,l1
0,necessary,необходимые,mal nécessaire
1,necessary,нужно,mal nécessaire
2,necessary,требуется,mal nécessaire
3,necessary,надо,mal nécessaire
4,necessary,обязательно,si nécessaire


In [196]:
azure_merge_dct = reference_to_dict(az_df)

In [197]:
df_azure_ctx_merge_approach_eval = eval_dicts(ref_dct,azure_merge_dct)


letter
plate
dinner
necessary
=======DIRECT SIM FINISHED========
[] []
необходимый необходимые 0.8005597775892018
l1_word_sim worked
('необходимый', 'nécessaire') ('необходимые', 'mal nécessaire')
no_match
('необходимый', 'nécessaire') ('необходимые', 'si nécessaire')
no_match
('необходимый', 'nécessaire') ('необходимые', 'nécessaire')
match_found
('необходимый', 'nécessaire') ('необходимые', 'inévitable')
no_match
необходимый нужно 0.4315309861780887
необходимый требуется 0.49444536840196773
необходимый надо -0.02096290929579834
необходимый обязательно 0.4242186320618566
['необходимый_nécessaire'] ['необходимые_nécessaire']
{'необходимый_nécessaire'} 1.0
{'необходимые_nécessaire'} eval_intersect 1 all_var_ev_dict_pairs_set 8 0.125
****************

inevitable
secret
*********** секретный ***********
=======DIRECT SIM FINISHED========
[] []
секрет секретный 0.41197042579780596
секрет тайной 0.2732048008482549
секрет втихомолку 0.10370884680483165
секрет потайной 0.16344850696090404
тай

*********** ломать ***********
=======DIRECT SIM FINISHED========
[] []
пауза перерыв 0.6285581723042439
l1_word_sim worked
('пауза', 'pause ') ('перерыв', 'félure')
no_match
пауза разрыв 0.1812551589082102
пауза отдыха 0.08320902448168109
пауза разорвать 0.09546117204332788
пауза сломать 0.03200472042180227
пауза нарушить 0.047977946422823874
пауза прорваться -0.010393790788377356
пауза ломать -0.009212024428250143
пауза каникулы 0.18810346962265673
пауза no_equality -0.055301054590672714
пауза брейк 0.32943845282191264
пауза пролом 0.06778241157419182
поломка, скол перерыв 0.17345833437513572
поломка, скол разрыв 0.42402309256711124
поломка, скол отдыха -0.10178465257588991
поломка, скол разорвать 0.14316102315848028
поломка, скол сломать 0.3162484806225587
поломка, скол нарушить 0.16133141126508457
поломка, скол прорваться 0.05625785610859312
поломка, скол ломать 0.13170891605267165
поломка, скол каникулы -0.07823676130075845
поломка, скол no_equality -0.1285848824371116
поломка, ск

In [200]:
df_azure_ctx_merge_approach_eval.head(30)

,eng_word,reference_coverage,paired_eval_pairs
0,necessary,1.000000,0.125000
1,secret,0.000000,0.000000
2,table,0.000000,0.000000
3,warm,0.333333,0.500000
4,America,0.000000,0.000000
5,entire,0.000000,0.000000
6,Belgium,0.000000,0.000000
7,background,0.000000,0.000000
8,child,1.000000,0.250000
9,childhood,0.000000,0.000000


In [199]:
get_overall_evaluation(df_azure_ctx_merge_approach_eval)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 3 columns):
eng_word              19 non-null object
reference_coverage    19 non-null float64
paired_eval_pairs     19 non-null float64
dtypes: float64(2), object(1)
memory usage: 536.0+ bytes
None


0.0

# CROSS APPROACH EVAL

In [173]:
cross_df = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/cross_approach.csv")
cross_df.head()

,Unnamed: 0,eng_word,rus,fr
0,0,for,для,pour
1,1,for,для,de
2,2,for,для,à
3,3,for,для,aux
4,4,for,для,par


In [174]:
del cross_df['Unnamed: 0']

In [175]:
cross_df.head()

,eng_word,rus,fr
0,for,для,pour
1,for,для,de
2,for,для,à
3,for,для,aux
4,for,для,par


In [176]:
cross_approach_dct = reference_to_dict(cross_df)

In [177]:
df_cross_approach_eval = eval_dicts(ref_dct,cross_approach_dct)


letter
*********** письмо ***********
*********** буква ***********
=======DIRECT SIM FINISHED========
['письмо_lettre', 'буква_lettre'] ['письмо_lettre', 'буква_lettre']
['письмо_lettre', 'буква_lettre'] ['письмо_lettre', 'буква_lettre']
{'буква_lettre', 'письмо_lettre'} 0.6666666666666666
{'буква_lettre', 'письмо_lettre'} eval_intersect 2 all_var_ev_dict_pairs_set 11 0.18181818181818182
****************

plate
*********** тарелка ***********
=======DIRECT SIM FINISHED========
['тарелка_assiette'] ['тарелка_assiette']
['тарелка_assiette'] ['тарелка_assiette']
{'тарелка_assiette'} 0.5
{'тарелка_assiette'} eval_intersect 1 all_var_ev_dict_pairs_set 15 0.06666666666666667
****************

dinner
*********** обед ***********
=======DIRECT SIM FINISHED========
['обед_déjeuner'] ['обед_déjeuner']
ужин обед 0.8418325624220971
l1_word_sim worked
('ужин', 'dîner') ('обед', 'dîner')
match_found
('ужин', 'dîner') ('обед', 'souper')
no_match
('ужин', 'dîner') ('обед', 'repas')
no_match
('ужин', 

момент точка зрения 0.2264654103411362
момент указывать -0.05534983325502369
момент дело 0.16759140993932545
момент отмечать -0.05306857996417568
момент показать 0.2002429089476438
момент подчеркивать -0.10479085374242847
момент навести -0.0631235725219643
момент точечный -0.015080269018710589
суть, смысл точка 0.16182616494235277
суть, смысл очко -0.025996453067187394
суть, смысл место -0.03542505861124043
суть, смысл вопрос 0.1301287113094764
суть, смысл пойнт -0.053813095313385746
суть, смысл смысл 0.8695020864800758
l1_word_sim worked
('суть, смысл', 'intérêt') ('смысл', 'point')
no_match
('суть, смысл', 'intérêt') ('смысл', 'but')
no_match
('суть, смысл', 'intérêt') ('смысл', 'intérêt')
match_found
('суть, смысл', 'intérêt') ('смысл', 'raison')
no_match
('суть, смысл', 'intérêt') ('смысл', 'sens')
no_match
суть, смысл этап 0.07045221616242811
суть, смысл точка зрения 0.19011683127629364
суть, смысл указывать 0.06900863739282873
суть, смысл дело 0.2175178752653471
суть, смысл отмеч

пауза преодолеть -0.00832876984561691
пауза отколоться 0.01188328745575978
пауза вырывать 0.013098914585112632
пауза преломлять -0.1023441923920684
пауза перерыв 0.6285581723042439
l1_word_sim worked
('пауза', 'pause ') ('перерыв', 'pause')
match_found
('пауза', 'pause ') ('перерыв', 'break')
no_match
('пауза', 'pause ') ('перерыв', 'interruption')
no_match
пауза разрыв 0.1812551589082102
пауза пролом 0.06778241157419182
пауза каникулы 0.18810346962265673
пауза передышка 0.6218265239751268
l1_word_sim worked
('пауза', 'pause ') ('передышка', 'no_equality')
no_match
поломка, скол ломать 0.13170891605267165
поломка, скол нарушать 0.052223721683750605
поломка, скол разбивать 0.18703596642484074
поломка, скол разорвать 0.14316102315848028
поломка, скол прорваться 0.05625785610859312
поломка, скол разрушить 0.13812633313218625
поломка, скол вырваться -0.09909499317140229
поломка, скол разбиться 0.18266259194589887
поломка, скол разразиться -0.017435012427793998
поломка, скол пробить 0.22001

In [185]:
get_overall_evaluation(df_cross_approach_eval)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
eng_word              25 non-null object
reference_coverage    25 non-null float64
paired_eval_pairs     25 non-null float64
dtypes: float64(2), object(1)
memory usage: 680.0+ bytes
None


0.6181818181818182

In [178]:
df_cross_approach_eval.head()

,eng_word,reference_coverage,paired_eval_pairs
0,letter,0.666667,0.181818
1,plate,0.500000,0.066667
2,dinner,1.000000,0.272727
3,necessary,1.000000,0.100000
4,secret,1.000000,0.230769


# CROSS APPROACH YANDEX CS YANDEX


In [206]:
cross_df_YY= pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/cross_approach_yandex_vs_yandex.csv")
cross_df_YY.head()


,Unnamed: 0,eng_word,rus,fr
0,0,for,для,pour
1,1,for,ради,pour
2,2,for,в течение,no_equality
3,3,for,в отношении,no_equality
4,4,for,для целей,no_equality


In [207]:
del cross_df_YY['Unnamed: 0']

In [208]:
cross_approach_YY_dct = reference_to_dict(cross_df_YY)

In [209]:
df_cross_approach_YY_eval = eval_dicts(ref_dct,cross_approach_YY_dct)


letter
*********** письмо ***********
*********** буква ***********
=======DIRECT SIM FINISHED========
['письмо_lettre', 'буква_lettre'] ['письмо_lettre', 'буква_lettre']
['письмо_lettre', 'буква_lettre'] ['письмо_lettre', 'буква_lettre']
{'буква_lettre', 'письмо_lettre'} 0.6666666666666666
{'буква_lettre', 'письмо_lettre'} eval_intersect 2 all_var_ev_dict_pairs_set 8 0.25
****************

plate
*********** тарелка ***********
=======DIRECT SIM FINISHED========
[] []
[] []
set() 0.0
set() eval_intersect 0 all_var_ev_dict_pairs_set 14 0.0
****************

dinner
*********** обед ***********
=======DIRECT SIM FINISHED========
[] []
ужин обед 0.8418325624220971
l1_word_sim worked
('ужин', 'dîner') ('обед', 'dîner')
match_found
ужин торжественный обед 0.636520567784271
l1_word_sim worked
('ужин', 'dîner') ('торжественный обед', 'no_equality')
no_match
ужин обеденный 0.26815043678990347
ужин званый 0.27428439393095644
['ужин_dîner'] ['обед_dîner']
{'ужин_dîner'} 0.5
{'обед_dîner'} eval_in

запятая (в числах) очко 0.09996512543258905
запятая (в числах) место 0.11612926225615797
запятая (в числах) вопрос 0.1125583509686064
запятая (в числах) пойнт -0.021064026776184318
запятая (в числах) смысл 0.12768242972620014
запятая (в числах) этап 0.057759194235369314
запятая (в числах) точка зрения 0.15865272342957107
запятая (в числах) указывать 0.20805186803341488
запятая (в числах) дело -0.06239311262557192
запятая (в числах) отмечать 0.007625528444579791
запятая (в числах) показать 0.06885374813291872
запятая (в числах) подчеркивать 0.02398588304777031
запятая (в числах) навести 0.037427150801877924
запятая (в числах) точечный 0.06207347530315356
показывать точка -0.07146897915999614
показывать очко -0.05904986754122569
показывать место -0.18958334425842968
показывать вопрос -0.02039794752143561
показывать пойнт -0.007604416540310881
показывать смысл -0.05226358981569865
показывать этап -0.1260335341330459
показывать точка зрения -0.05299245138955073
показывать указывать 0.64347

In [210]:
get_overall_evaluation(df_cross_approach_YY_eval)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 3 columns):
eng_word              25 non-null object
reference_coverage    25 non-null float64
paired_eval_pairs     25 non-null float64
dtypes: float64(2), object(1)
memory usage: 680.0+ bytes
None


0.4